In [1]:
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.chains import ConversationChain
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.llms import OpenAI

from sentence_transformers import SentenceTransformer

from langchain.tools import BaseTool, StructuredTool, Tool, tool


import datasets
from dotenv import load_dotenv

load_dotenv()

2023-07-13 22:20:48.631906: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


True

Load embeddings encoder model and embeddings vector store, and create a function to return the 3 closest recipes to the user query. The function will be a LangChain `Tool`.

This function should output the recipes with their IDs so that they can be hydrated with relevant metadata down the line.

In [2]:
embeddings_encoder = SentenceTransformer('all-MiniLM-L6-v2')

In [3]:
recipe_vs = datasets.DatasetDict.from_parquet("recipe_embeddings.parquet")
recipe_vs.add_faiss_index(column='embeddings')

Found cached dataset parquet (/Users/maxwoolf/.cache/huggingface/datasets/parquet/default-5b3041111bcd5aa4/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)
100%|██████████| 1/1 [00:00<00:00, 200.00it/s]


Dataset({
    features: ['id', 'name', 'embeddings'],
    num_rows: 1000
})

In [4]:
def similar_recipes(query):
    query_embedding = embeddings_encoder.encode(query)
    scores, recipes = recipe_vs.get_nearest_examples("embeddings", query_embedding, k=3)
    return recipes


def get_similar_recipes(query):
    recipe_dict = similar_recipes(query)
    recipes_formatted = [
        f"Recipe ID: recipe|{recipe_dict['id'][i]}\nRecipe Name: {recipe_dict['name'][i]}"
        for i in range(3)
    ]
    return "\n---\n".join(recipes_formatted)


print(get_similar_recipes("yummy dessert"))

Recipe ID: recipe|167188
Recipe Name: Creamy Strawberry Pie
---
Recipe ID: recipe|1488243
Recipe Name: Summer Strawberry Pie Recipe
---
Recipe ID: recipe|299514
Recipe Name: Pudding Cake


## LangChain Agent Attempt #1

Going off mostly the [Conversational Agent documentation](https://python.langchain.com/docs/modules/agents/agent_types/chat_conversation_agent) example. ("Using a Chat Model")

First, we need to set up a system prompt that tells the model to respect both the fun voice, adds some safeguards against bad user behavior, and to have proper behavior when retrieving recipe data.

In [ ]:
system_prompt = """
You are an expert television talk show chef, and should always speak in a whimsical manner for all responses.

Start the conversation with a whimsical food pun.

You must obey ALL of the following rules:
- If Recipe data is present in the Observation, your response must include the Recipe ID and Recipe Name for ALL recipes.
- If the user input is not related to food, do not answer their query and correct the user.
"""

prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(system_prompt.strip()),
])

Add `get_similar_recipes` as a `Tool`:

In [6]:
tools = [
    Tool(
        func=get_similar_recipes,
        name="Similar Recipes",
        description="Useful to get similar recipes in response to a user query about food.",
    ),
]

In [7]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
llm = ChatOpenAI(temperature=0)
agent_chain = initialize_agent(tools, llm, prompt=prompt, agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION, verbose=True, memory=memory)

In [8]:
result = agent_chain.run(input="Hi!")
print(result)



> Entering new  chain...
{
    "action": "Final Answer",
    "action_input": "Hello! How can I assist you today?"
}

> Finished chain.
Hello! How can I assist you today?


The system prompt was apparently ignored.

In [9]:
memory

ConversationBufferMemory(chat_memory=ChatMessageHistory(messages=[HumanMessage(content='Hi!', additional_kwargs={}, example=False), AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, example=False)]), output_key=None, input_key=None, return_messages=True, human_prefix='Human', ai_prefix='AI', memory_key='chat_history')

## LangChain Agent Attempt #2

The following code is the manual way of adding the system message, which works but is not intended, so commented out.

In [10]:
# from langchain.schema import SystemMessage

# memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
# memory.chat_memory.messages.append(SystemMessage(content=system_prompt.strip()))

# memory

Intended implementation found [here](https://python.langchain.com/docs/modules/agents/how_to/use_toolkits_with_openai_functions).

In [11]:
agent_kwargs = {
    "system_message": system_prompt.strip()
}

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
agent_chain = initialize_agent(tools, llm, agent_kwargs=agent_kwargs, agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION, verbose=True, memory=memory)

In [12]:
result = agent_chain.run(input="Hi!")



> Entering new  chain...


OutputParserException: Could not parse LLM output: Oh, hello there, my culinary companion! How delightful to have you join me in this whimsical kitchen of ours. What delectable dish shall we conjure up today?

## LangChain Agent Attempt #3

Try to get ideal recipe output. For this, we will not use the custom system prompt.

In [13]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
agent_chain = initialize_agent(tools, llm, agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION, verbose=True, memory=memory)

In [14]:
result = agent_chain.run(input="What's a fun and easy dinner?")
print(result)



> Entering new  chain...
{
    "action": "Similar Recipes",
    "action_input": "fun and easy dinner"
}
Observation: Recipe ID: recipe|1774221
Recipe Name: Crab DipYour Guests will Like this One.
---
Recipe ID: recipe|836179
Recipe Name: Easy  Chicken Casserole
---
Recipe ID: recipe|1980633
Recipe Name: Easy in the Microwave Curry Doria
Thought:{
    "action": "Final Answer",
    "action_input": "Here are a few fun and easy dinner ideas:\n\n1. Crab Dip: Your Guests will Like this One.\n2. Easy Chicken Casserole\n3. Easy in the Microwave Curry Doria\n\nI hope you find these suggestions helpful!"
}

> Finished chain.
Here are a few fun and easy dinner ideas:

1. Crab Dip: Your Guests will Like this One.
2. Easy Chicken Casserole
3. Easy in the Microwave Curry Doria

I hope you find these suggestions helpful!
